In [1]:
import sys
sys.path.append('f:/Projects/PcAnalyse/')

import datetime
import tensorflow as tf
import tensorflow.feature_column as fc 
tf.logging.set_verbosity(tf.logging.INFO)

import numpy as np
import StockDataLoad as sdl
import StockLearningData as sld
import StockPredict as sp

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 30, 20, 1])

  # Convolutional Layer #1
  # Output shape is [-1, 30, 20, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # Output shape is [-1, 15, 10, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  # Output shape is [-1, 15, 10, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  # Output shape is [-1, 5, 5, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[3, 2], strides=[3,2])

  # Dense Layer
  # Output shape is [-1, 1600]
  pool2_flat = tf.reshape(pool2, [-1, 5 * 5 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  # Output shape is [-1, 1024]
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=2)
#   logits = tf.layers.dense(inputs=dense, units=2)

#   print(logits)
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
stock_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="F:/Projects/PcAnalyse/models/cnn-01")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'F:/Projects/PcAnalyse/models/cnn-01', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021C5AA245F8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [18]:
small_train_data, small_train_labels = sld.get_learning_data_from_path('d:/export-small')
# small_train_data = np.array(small_train_data)
# small_train_labels = np.array(small_train_labels)
# train_data, train_labels = sld.get_learning_data_from_path('d:/export', end_date='2017/12/30')
# train_data = np.array(train_data)
# train_labels = np.array(train_labels)

SH#600069.txt
SH#600070.txt


In [7]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
#     batch_size=32,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
stock_classifier.train(
    input_fn=train_input_fn,
    steps=5000,
    hooks=[logging_hook])

datetime	 fc	 np	 sdl	 sld	 sp	 sys	 tf	 


In [8]:
predict_data = sdl.load_path2dict('d:/export-small/', start_date='2018/01/04')

In [152]:
predict_data['600070'].head()

,open,high,low,close,volume,transation,ma5,ma20,v_ma5,v_ma10,open_pc,close_pc,volume_pc,ma5_pc,ma20_pc,v_ma5_pc,v_ma10_pc,rise_pc,amplitude,oc_pc
date,,,,,,,,,,,,,,,,,,,,
2018/01/04,9.55,9.60,9.42,9.58,1631702,15509115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019108,0.166667
2018/01/05,9.60,9.60,9.44,9.50,1736202,16508281.0,NaN,NaN,NaN,NaN,0.005236,-0.008351,0.064044,NaN,NaN,NaN,NaN,-0.005236,0.016949,0.625000
2018/01/08,9.48,9.48,9.34,9.38,1335505,12539461.0,NaN,NaN,NaN,NaN,-0.012500,-0.012632,-0.230789,NaN,NaN,NaN,NaN,-0.022917,0.014989,0.714286
2018/01/09,9.37,9.43,9.21,9.22,1873322,17390668.0,NaN,NaN,NaN,NaN,-0.011603,-0.017058,0.402707,NaN,NaN,NaN,NaN,-0.027426,0.023887,0.681818
2018/01/10,9.20,9.39,9.20,9.33,2340635,21815834.0,9.402,NaN,1783473.2,NaN,-0.018143,0.011931,0.249457,NaN,NaN,NaN,NaN,-0.004269,0.020652,0.684211


In [150]:
result = sp.get_predicted_df(stock_classifier, predict_data, '2018/06/04')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from F:/Projects/PcAnalyse/models/cnn-01\model.ckpt-16
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [8]:
print(result)

NameError: name 'result' is not defined

In [9]:
tf.logging.set_verbosity(tf.logging.ERROR)
sp.predict_stock_range(stock_classifier, predict_data, '2018/03/19', '2019/04/16')

datetime	 fc	 np	 sdl	 sld	 sp	 sys	 tf	 


==================================下面是DNN部分==================================

In [50]:
# small_train_data, small_train_labels = sld.get_learning_data_from_path('d:/export-small')
train_data, train_labels = sld.get_learning_data_from_path('d:/export', end_date='2017/12/30')

In [93]:
features_cols = tf.contrib.learn.infer_real_valued_columns_from_input(train_data)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[512,256,128,64], n_classes=2, feature_columns=features_cols,
                                         model_dir="f:/Projects/PcAnalyse/models/DNN-512-256-128-64-nc2-ma5",)
dnn_clf.fit(train_data, train_labels, batch_size=500, steps=5000)
# dnn_clf.fit(train_data, train_labels, steps=5000)

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x0000025DD668C1D0>, 'hidden_units': [512, 256, 128, 64], 'feature_columns': (_RealValuedColumn(column_name='', dimension=540, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x0000025DFED85598>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [55]:
# predict_data = sdl.load_path2dict('d:/export/', start_date='2018/01/04')

In [94]:
tf.logging.set_verbosity(tf.logging.ERROR)
sp.predict_stock_range(dnn_clf, predict_data, '2018/03/22', '2019/04/19')

2018/03/22
  600031 1 0.5071841478347778	 -0.12484394506866152
  600131 1 0.5057438611984253	 1.7350157728706674
  600536 1 0.5178360342979431	 13.705234159779629
  600570 1 0.5066242814064026	 9.586905689789559
  600593 1 0.519586443901062	 0.0
  600654 1 0.5027448534965515	 -4.024767801857582
  600763 1 0.5188202857971191	 -4.380593499764482
  600776 1 0.516981303691864	 0.4754358161648217
  600801 1 0.5178903341293335	 -1.5488215488215518
  603043 1 0.518189549446106	 -0.5540166204986228
  603338 1 0.5002033710479736	 -5.240963855421686
  603383 1 0.5186229944229126	 1.594746716697941
  603605 1 0.515366792678833	 2.4851242562128135
  603676 1 0.5124225616455078	 7.949308755760362
  000338 1 0.502269983291626	 2.7595269382391585
  000550 1 0.5187439322471619	 -7.470588235294115
  000596 1 0.5193899869918823	 -2.5181679905357477
  000789 1 0.5137158632278442	 -1.9014693171996597
  000860 1 0.5178277492523193	 3.144329896907232
  002032 1 0.5187735557556152	 6.196038598273229
  002124

  002864 1 0.5189961791038513	 21.09375
  002912 1 0.5150639414787292	 6.257787622871377
  002916 1 0.5109807848930359	 11.740192556401784
  300012 1 0.5180662274360657	 2.087682672233813
  300033 1 0.5177125334739685	 2.0953163516844766
  300107 1 0.5178104639053345	 2.6402640264026425
  300253 1 0.512035071849823	 1.7497812773403418
  300348 1 0.5117097496986389	 74.42099845599589
  300572 1 0.5110049247741699	 -0.20308692120227745
  300663 1 0.5171581506729126	 2.918973326623041
  300684 1 0.5195003747940063	 30.70915338956578
  300725 1 0.5203558206558228	 2.0399490012749646
2018/03/29
  600074 1 0.5193918347358704	 0.0
  600131 1 0.5122565627098083	 -1.2441679626749622
  600536 1 0.5172423720359802	 4.229797979797991
  600593 1 0.5186511278152466	 -1.1353711790392926
  600654 1 0.5077235102653503	 -0.3257328990227944
  600763 1 0.5198516249656677	 4.003040283759813
  600776 1 0.5169586539268494	 0.8038585209003187
  600801 1 0.5191181898117065	 0.6872852233676952
  600810 1 0.5012

  002463 1 0.5066298842430115	 -1.6746411483253445
  002760 1 0.5118569135665894	 -1.9404915912031169
  002864 1 0.5060170888900757	 -9.05838580423583
  002916 1 0.5201420187950134	 -1.8110621634850759
  300012 1 0.5190036296844482	 1.4522821576763358
  300033 1 0.5188085436820984	 -1.0157339175463016
  300107 1 0.5178510546684265	 -1.0604453870625625
  300348 1 0.5014917254447937	 68.18858560794047
  300596 1 0.5008364319801331	 -0.43689320388349445
  300663 1 0.5175796747207642	 -1.27420998980632
  300684 1 0.5194543600082397	 -10.15675327464032
  300725 1 0.5199354887008667	 1.8132975151108048
2018/04/09
  600074 1 0.5177966356277466	 -20.52631578947368
  600131 1 0.5025317072868347	 -5.546492659053832
  600198 1 0.5175217986106873	 8.758169934640522
  600536 1 0.5175603628158569	 -1.9596199524940516
  600593 1 0.514416515827179	 -0.30823425803610865
  600763 1 0.5087102651596069	 -0.07334963325183652
  600776 1 0.5170730352401733	 -2.6521060842433686
  600801 1 0.5113167762756348	 

  002760 1 0.5181689858436584	 -0.0896860986547066
  002916 1 0.5201564431190491	 2.5063567017798682
  300012 1 0.5172991156578064	 1.414141414141402
  300033 1 0.513472318649292	 -0.4680504680504761
  300107 1 0.5181170701980591	 2.572347266881031
  300110 1 0.5170615315437317	 -6.271186440677968
  300394 1 0.5080607533454895	 0.7614213197969651
  300638 1 0.5001758337020874	 -1.0881801125703534
  300663 1 0.5173658728599548	 -10.957758260142205
  300684 1 0.5159129500389099	 11.146575006137999
  300725 1 0.5117950439453125	 -1.577418015774181
2018/04/16
  600074 1 0.517708957195282	 -7.361963190184043
  600198 1 0.5187170505523682	 6.875000000000009
  600536 1 0.5178256630897522	 5.584045584045586
  600763 1 0.5058959722518921	 -0.06976744186046777
  600776 1 0.5172461867332458	 2.931596091205222
  600810 1 0.5188484191894531	 -6.134969325153371
  601003 1 0.5030733942985535	 11.33004926108375
  603286 1 0.5183036923408508	 -3.6278735632183965
  603605 1 0.5179455280303955	 0.9256580

  002565 1 0.5097330808639526	 -2.185792349726778
  002760 1 0.5174697041511536	 1.6981132075471672
  002916 1 0.5168716311454773	 -7.3437107707757905
  300012 1 0.5013952255249023	 -1.6427104722792623
  300033 1 0.511378824710846	 3.4038054968287517
  300107 1 0.5194566249847412	 79.25133689839575
  300110 1 0.5180895328521729	 -2.1238938053097365
  300394 1 0.5183997750282288	 -1.6393442622950813
  300663 1 0.5172022581100464	 14.890438247011964
  300725 1 0.506613552570343	 8.58987836011413
2018/04/23
  600074 1 0.5178425908088684	 -2.5806451612903247
  600131 1 0.5017377734184265	 3.39285714285715
  600198 1 0.5187370181083679	 -3.1710079275198115
  600536 1 0.5170812606811523	 9.153633854645824
  600654 1 0.5054782032966614	 4.761904761904759
  600763 1 0.5150063037872314	 12.871027055424216
  600776 1 0.5172263979911804	 -1.705426356589152
  600801 1 0.502088189125061	 0.21754894851342377
  600810 1 0.5190274119377136	 24.893162393162395
  603088 1 0.5084228515625	 3.351614868982

...

2019/04/18
  600010 1 0.5193812251091003	 1.0869565217391313
  600012 1 0.518321692943573	 0.13986013986013687
  600022 1 0.520004153251648	 1.5544041450777217
  600056 1 0.5047484040260315	 0.8222643896268121
  600060 1 0.5174689292907715	 1.6559337626495079
  600062 1 0.5196417570114136	 0.6271777003484311
  600066 1 0.5193804502487183	 5.08250825082508
  600069 1 0.5173368453979492	 -0.25062656641604586
  600074 1 0.5173115730285645	 -3.418803418803422
  600076 1 0.5097895860671997	 0.38387715930903
  600080 1 0.5176095962524414	 0.23255813953487878
  600084 1 0.5054994821548462	 -3.3802816901408357
  600090 1 0.5105875134468079	 0.15873015873015536
  600107 1 0.5200940370559692	 5.222222222222229
  600110 1 0.5178959369659424	 4.725897920604915
  600119 1 0.5199902057647705	 -2.2727272727272694
  600121 1 0.5179905295372009	 4.477611940298523
  600122 1 0.516385555267334	 1.2345679012345598
  600130 1 0.5186623334884644	 -1.0330578512396658
  600136 1 0.5104116797447205	 10.0
  600

  000926 1 0.5202749371528625	 0.6281407035175857
  000933 1 0.517424464225769	 2.5362318840579814
  000939 1 0.5172141790390015	 -3.418803418803422
  000949 1 0.515129029750824	 0.5763688760806921
  000953 1 0.5195415019989014	 -1.2048192771084436
  000963 1 0.519745945930481	 4.822256568778965
  000972 1 0.5106024146080017	 0.8042895442359197
  000976 1 0.5199224352836609	 0.16611295681064245
  000980 1 0.5171907544136047	 3.937007874015748
  000981 1 0.5179075002670288	 -1.746724890829696
  000982 1 0.515555739402771	 5.238095238095232
  000983 1 0.5189169645309448	 7.369942196531789
  000989 1 0.5181183815002441	 2.9862792574656916
  000995 1 0.5182151794433594	 5.055487053020964
  000998 1 0.5196744203567505	 0.7236842105263238
  001896 1 0.5197051763534546	 -0.23923444976076047
  002009 1 0.519571840763092	 2.586975914362169
  002010 1 0.5196740031242371	 0.2136752136752281
  002012 1 0.5059584975242615	 3.3093525179856047
  002018 1 0.5129461288452148	 -2.2388059701492558
  0020

2019/04/19
  600010 1 0.5192924737930298	 0.0
  600012 1 0.5182687640190125	 0.0
  600022 1 0.519838273525238	 0.0
  600056 1 0.5021556615829468	 0.0
  600060 1 0.5198145508766174	 0.0
  600062 1 0.5195226073265076	 0.0
  600066 1 0.5158930420875549	 0.0
  600069 1 0.5173243880271912	 0.0
  600074 1 0.5172157883644104	 0.0
  600076 1 0.5099754929542542	 0.0
  600080 1 0.5087651014328003	 0.0
  600084 1 0.5088699460029602	 0.0
  600090 1 0.5047582387924194	 0.0
  600107 1 0.5199097394943237	 0.0
  600110 1 0.5176631808280945	 0.0
  600114 1 0.5113615393638611	 0.0
  600119 1 0.5201137661933899	 0.0
  600121 1 0.518059492111206	 0.0
  600122 1 0.5162313580513	 0.0
  600130 1 0.5185941457748413	 0.0
  600136 1 0.5203213691711426	 0.0
  600139 1 0.5195268988609314	 0.0
  600146 1 0.5177026391029358	 0.0
  600165 1 0.5166167616844177	 0.0
  600172 1 0.5162851214408875	 0.0
  600175 1 0.5159403085708618	 0.0
  600179 1 0.5177803039550781	 0.0
  600188 1 0.51873779296875	 0.0
  600191 1 0.519

  000636 1 0.505150556564331	 0.0
  000663 1 0.5174066424369812	 0.0
  000673 1 0.5142250061035156	 0.0
  000697 1 0.5202303528785706	 0.0
  000700 1 0.50785893201828	 0.0
  000711 1 0.5040135383605957	 0.0
  000725 1 0.5193780064582825	 0.0
  000732 1 0.5187491774559021	 0.0
  000751 1 0.5199316143989563	 0.0
  000752 1 0.5199715495109558	 0.0
  000755 1 0.5019041895866394	 0.0
  000760 1 0.5196243524551392	 0.0
  000762 1 0.5183203816413879	 0.0
  000782 1 0.5203154683113098	 0.0
  000792 1 0.5181739330291748	 0.0
  000793 1 0.5171459317207336	 0.0
  000797 1 0.5096367597579956	 0.0
  000807 1 0.5185261368751526	 0.0
  000815 1 0.5176313519477844	 0.0
  000820 1 0.5162811875343323	 0.0
  000829 1 0.5163893103599548	 0.0
  000835 1 0.5187101364135742	 0.0
  000838 1 0.5196617245674133	 0.0
  000839 1 0.5140218138694763	 0.0
  000868 1 0.5176458358764648	 0.0
  000885 1 0.5191535949707031	 0.0
  000887 1 0.5156117677688599	 0.0
  000892 1 0.5198214650154114	 0.0
  000897 1 0.5142775177

  300355 1 0.5154787302017212	 0.0
  300362 1 0.5158456563949585	 0.0
  300364 1 0.5163923501968384	 0.0
  300366 1 0.5166876912117004	 0.0
  300367 1 0.519491970539093	 0.0
  300373 1 0.5192020535469055	 0.0
  300376 1 0.5101053714752197	 0.0
  300393 1 0.5184810757637024	 0.0
  300401 1 0.5183228254318237	 0.0
  300409 1 0.5196017622947693	 0.0
  300410 1 0.5076055526733398	 0.0
  300415 1 0.5181222558021545	 0.0
  300418 1 0.5180103778839111	 0.0
  300422 1 0.5195460319519043	 0.0
  300426 1 0.5155462026596069	 0.0
  300430 1 0.5192751288414001	 0.0
  300431 1 0.5148499608039856	 0.0
  300432 1 0.5149308443069458	 0.0
  300433 1 0.5159044861793518	 0.0
  300441 1 0.5198466181755066	 0.0
  300447 1 0.5186994671821594	 0.0
  300460 1 0.5064706802368164	 0.0
  300462 1 0.5182390213012695	 0.0
  300463 1 0.500434935092926	 0.0
  300466 1 0.5181846618652344	 0.0
  300467 1 0.5188537836074829	 0.0
  300477 1 0.519184410572052	 0.0
  300485 1 0.5173737406730652	 0.0
  300494 1 0.5144667625

In [53]:
predict_data['600145'].tail()

,open,high,low,close,volume,transation,ma5,ma20,v_ma5,v_ma10,open_pc,close_pc,volume_pc,ma5_pc,ma20_pc,v_ma5_pc,v_ma10_pc,rise_pc,amplitude,oc_pc
date,,,,,,,,,,,,,,,,,,,,


In [7]:
import matplotlib.pyplot as plt

In [3]:
df = sdl.load_path2dict('d:/export-small')

In [24]:
train_labels[:10]

array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0])

In [8]:
plt.show()